In [74]:
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import os

In [2]:
def create_regressor_columns_string(columns):
  # Join all independent variables to define the formula used by the model
  regressor_columns = list(filter(lambda x: x != 'impression_count',columns))
  regressor_columns_string = "+".join(regressor_columns)
  return regressor_columns_string

In [79]:
def regression(path):
    # Load data
    regression_df_pd = pd.read_csv(path)
    regression_df_pd = regression_df_pd.drop('tweet_text', axis=1)
    
    # Apply log transformation to independent variables
    for var in regression_df_pd.columns:
            regression_df_pd[var] = regression_df_pd[var].apply(lambda x: np.log(1+x))

    # Define formula without cross terms
    regressor_columns_string = create_regressor_columns_string(regression_df_pd.columns)

    # Fit model
    mod = smf.ols(formula=f'impression_count ~ {regressor_columns_string}', data=regression_df_pd)
    res = mod.fit()

    # Get p-values
    p_values = res.pvalues
    nb_variables_before = len(p_values)
    
    # Filter p-values < 0.05
    p_values = p_values[p_values < 0.05]
    nb_variables_after = len(p_values)

    # Get significant variables
    significant_variables = list(p_values.index)
    # Remove intercept if significant
    try:
        significant_variables.remove('Intercept')
    except:
        pass

    print(f'Number of discarded variables: {nb_variables_before - nb_variables_after}')
    print(f'Significant variables ({len(significant_variables)}): {significant_variables}')

    # Perform new regression with significant variables
    regressor_columns_string_new = create_regressor_columns_string(significant_variables)
    mod_new = smf.ols(formula=f'impression_count ~ {regressor_columns_string_new}', data=regression_df_pd)
    res_new = mod_new.fit()

    # Save res as html
    res_html = res.summary().as_html()
    name = path.split('/')[-1].split('.')[0][:-3]
    path = os.path.join("../data/regression/html_regression", name, ".html")
    with open(path, 'w') as f:
        f.write(res_html)

    # Save res_new as html
    res_new_html = res_new.summary().as_html()
    path_opti = os.path.join("../data/regression/html_regression", name+"_opti.html")
    with open(path_opti, 'w') as f:
        f.write(res_html)

    return res, res_new

In [80]:
path_french_celebrities = '../data/regression/french_celebrities_regression_df.csv'
res_french_celebrities = regression(path_french_celebrities)

display(res_french_celebrities[0].summary())
display(res_french_celebrities[1].summary())

Number of discarded variables: 10
Significant variables (14): ['dummy_entertainment', 'dummy_entertainmentleisurebusiness', 'dummy_francepolitics', 'dummy_jeanlucmlenchon', 'dummy_nikosaliagas', 'dummy_politicalfigures', 'dummy_staderennaisfc', 'hashtags_count', 'mentions_count', 'tweet_external_urls_count', 'tweet_length', 'tweet_medias_count', 'tweet_sentiment', 'followers_count']


FileNotFoundError: [Errno 2] No such file or directory: '../data/regression/html_regression/french_celebrities_regression/.html'

In [63]:
path_us_celebrities = '../data/regression/american_celebrities_regression_df.csv'
res_french_celebrities = regression(path_us_celebrities)

display(res_french_celebrities[0].summary())
display(res_french_celebrities[1].summary())

Number of discarded variables: 19
Significant variables (36): ['dummy_americanfootball', 'dummy_barstoolsports', 'dummy_businesspersonalities', 'dummy_celebrities', 'dummy_competitionshows', 'dummy_entertainment', 'dummy_events', 'dummy_famouscomedians', 'dummy_gamingbusiness', 'dummy_jaketapper', 'dummy_johncusack', 'dummy_kenjeong', 'dummy_lebronjames', 'dummy_music', 'dummy_nba', 'dummy_nbabasketball', 'dummy_nbaplayers', 'dummy_news', 'dummy_pftcommenter', 'dummy_politics', 'dummy_pop', 'dummy_sportsfigures', 'dummy_talkshows', 'dummy_technologybusiness', 'dummy_thomassanders', 'dummy_tvmoviesrelatedentertainment', 'dummy_tweet_period_afternoon', 'dummy_tweet_period_night', 'dummy_wizkhalifa', 'dummy_youtubers', 'mentions_count', 'tweet_external_urls_count', 'tweet_length', 'tweet_medias_count', 'tweet_sentiment', 'followers_count']


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       impression_count   R-squared:                       0.549
Model:                            OLS   Adj. R-squared:                  0.548
Method:                 Least Squares   F-statistic:                     603.0
Date:                Sun, 14 May 2023   Prob (F-statistic):               0.00
Time:                        16:08:01   Log-Likelihood:                -73150.
No. Observations:               26825   AIC:                         1.464e+05
Df Residuals:                   26770   BIC:                         1.469e+05
Df Model:                          54                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              1.0914      0.275      3.975      0.000       0.553       1.630
dummy_americanfootball                 2.6065      0.228     11.408      0.000       2.159       3.054
dummy_barstoolsports                   6.1525      1.783      3.451      0.001       2.658       9.647
dummy_basketball                       0.4776      0.365      1.310      0.190      -0.237       1.192
dummy_businessfinance                 -0.2959      0.621     -0.477      0.634      -1.513       0.921
dummy_businesspersonalities            1.3596      0.241      5.643      0.000       0.887       1.832
dummy_celebrities                      2.0258      0.151     13.452      0.000       1.731       2.321
dummy_competitionshows                 0.7256      0.338      2.144      0.032       0.062       1.389
dummy_digitalcreators                 -0.0541      0.637     -0.085      0.932      -1.303       1.195
dummy_elonmusk                         4.7426      2.733      1.735      0.083      -0.614      10.099
dummy_entertainment                   -0.6963      0.164     -4.243      0.000      -1.018      -0.375
dummy_entertainmentleisurebusiness    -0.2216      0.175     -1.268      0.205      -0.564       0.121
dummy_events                           0.5316      0.194      2.739      0.006       0.151       0.912
dummy_famouscomedians                  0.5733      0.137      4.172      0.000       0.304       0.843
dummy_financialservicesbusiness        0.2532      0.533      0.475      0.635      -0.792       1.298
dummy_foodbeveragebusiness             0.2901      0.202      1.435      0.151      -0.106       0.686
dummy_gamingbusiness                  -0.5880      0.203     -2.894      0.004      -0.986      -0.190
dummy_hiphop                          -2.3337      1.207     -1.934      0.053      -4.699       0.031
dummy_jaketapper                      -3.3020      0.330     -9.994      0.000      -3.950      -2.654
dummy_johncusack                      -4.2110      0.179    -23.577      0.000      -4.561      -3.861
dummy_journalists                     -0.1042      0.406     -0.257      0.797      -0.900       0.691
dummy_kenjeong                        -2.7170      0.202    -13.478      0.000      -3.112      -2.322
dummy_leadership                      -0.4020      2.699     -0.149      0.882      -5.692       4.888
dummy_lebronjames                     -5.0208      0.229    -21.932      0.000      -5.469      -4.572
dummy_moviestv                        -0.3999      0.259     -1.544      0.123      -0.908       0.108
dummy_music                            0.7276      0.230      3.160      0.002       0.276       1.179
dummy_nba                             -2.8671      0.684     -4.189      0.000      -4.209      

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       impression_count   R-squared:                       0.546
Model:                            OLS   Adj. R-squared:                  0.546
Method:                 Least Squares   F-statistic:                     895.3
Date:                Sun, 14 May 2023   Prob (F-statistic):               0.00
Time:                        16:08:01   Log-Likelihood:                -73229.
No. Observations:               26825   AIC:                         1.465e+05
Df Residuals:                   26788   BIC:                         1.468e+05
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              0.5314      0.311      1.708      0.088      -0.078       1.141
dummy_americanfootball                 2.5510      0.200     12.779      0.000       2.160       2.942
dummy_barstoolsports                   6.1596      1.787      3.447      0.001       2.658       9.662
dummy_businesspersonalities            2.3488      0.206     11.414      0.000       1.945       2.752
dummy_celebrities                      1.8885      0.145     13.044      0.000       1.605       2.172
dummy_competitionshows                 0.9518      0.201      4.732      0.000       0.558       1.346
dummy_entertainment                   -0.9081      0.140     -6.504      0.000      -1.182      -0.634
dummy_events                           0.5926      0.190      3.116      0.002       0.220       0.965
dummy_famouscomedians                  0.4533      0.136      3.343      0.001       0.188       0.719
dummy_gamingbusiness                  -0.5467      0.202     -2.702      0.007      -0.943      -0.150
dummy_jaketapper                      -3.4596      0.268    -12.932      0.000      -3.984      -2.935
dummy_johncusack                      -4.1356      0.178    -23.207      0.000      -4.485      -3.786
dummy_kenjeong                        -2.5999      0.201    -12.965      0.000      -2.993      -2.207
dummy_lebronjames                     -4.9802      0.228    -21.798      0.000      -5.428      -4.532
dummy_music                           -0.0043      0.162     -0.027      0.979      -0.321       0.312
dummy_nba                             -2.5155      0.590     -4.260      0.000      -3.673      -1.358
dummy_nbabasketball                    1.6896      0.571      2.956      0.003       0.569       2.810
dummy_nbaplayers                       2.8521      0.211     13.492      0.000       2.438       3.266
dummy_news                            -1.0554      0.214     -4.924      0.000      -1.475      -0.635
dummy_pftcommenter                    -3.5383      1.794     -1.973      0.049      -7.054      -0.022
dummy_politics                        -0.2938      0.123     -2.380      0.017      -0.536      -0.052
dummy_pop                             -1.3941      0.204     -6.845      0.000      -1.793      -0.995
dummy_sportsfigures                    2.3981      0.150     15.945      0.000       2.103       2.693
dummy_talkshows                        3.5512      0.196     18.155      0.000       3.168       3.935
dummy_technologybusiness               4.5575      0.242     18.867      0.000       4.084       5.031
dummy_thomassanders                    1.7096      0.677      2.526      0.012       0.383       3.036
dummy_tvmoviesrelatedentertainment     1.7889      0.231      7.728      0.000       1.335      